In [3]:
# Imports

import os
import re
from argparse import ArgumentParser
from glob import glob

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.utils.data as data
import pytorch_lightning as pl
from tqdm.auto import tqdm

import torchmetrics.functional as metrics

from dgl.nn import GraphConv
import dgllife as life
from dgllife.utils import ConcatFeaturizer
import datamol as dm
from multiprocessing import Manager
from rdkit.rdBase import BlockLogs
from rdkit import Chem
from rdkit.Chem import rdchem
import subprocess

import wandb

DEVICE = torch.device("cpu")

# Import GPU-related things
if torch.cuda.is_available():
    # import cupy as np
    # import cudf as pd

    # Ensure that all operations are deterministic on GPU (if used) for reproducibility
    torch.backends.cudnn.determinstic = True
    torch.backends.cudnn.benchmark = False

    DEVICE = torch.device("cuda:0")
# else:

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = os.environ.get("PATH_DATASETS", "data/")
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "saved_models/")

# Setting the seed
pl.seed_everything(42)

print('CUDA:', torch.cuda.is_available())
print("Device:", DEVICE)

temp_path = './temp'
data_path = './data'

if not os.path.exists(temp_path):
    os.mkdir(temp_path)

Global seed set to 42


CUDA: True
Device: cuda:0


In [4]:
xyz_filepath_list = list(glob(f'{data_path}/*.xyz'))

xyz_filepath_list.sort()
print('total xyz filepath # ', len(xyz_filepath_list))
xyz_filepath_list[0]

total xyz filepath #  133885


'./data/dsgdb9nsd_000001.xyz'

In [5]:
def in_ipython():
    try:
        return __IPYTHON__
    except NameError:
        return False

In [6]:
df = pd.read_csv('./data.csv')

In [13]:
gaff2_types = df['type'].unique().tolist()

In [14]:
gaff2_types

['c3',
 'hc',
 'n9',
 'hn',
 'oh',
 'ho',
 'n3',
 'h3',
 'os',
 'h2',
 'h1',
 'n8',
 'cx',
 'op',
 'n7',
 'np',
 'cy',
 'oq',
 'nz',
 'o',
 'hx',
 'n6',
 'n5',
 'f',
 'ny',
 'nq',
 'nx',
 'nk',
 'nl',
 'nh',
 'n2',
 'c2',
 'ha',
 'nu',
 'h4',
 'cu',
 'cv',
 'n4']

In [ ]:
class CustomDataset(data.Dataset):
    def __init__(self, X, y):
        super().__init__()
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
def get_subfeatures(num_features, divider):
    return int(np.floor(num_features / divider))

In [6]:
class FFNetwork(pl.LightningModule):
    def __init__(self, num_features, num_classes, dropout: float = 0.1):
        super().__init__()

        self.num_features = num_features
        self.num_classes = num_classes

        self.save_hyperparameters()

        self.sequential = nn.Sequential(
            GraphConv(self.num_features, get_subfeatures(self.num_features, 2)),
            nn.ReLU(),
            GraphConv(get_subfeatures(self.num_features, 2), self.num_classes)
        )

    def training_step(self, batch, batch_idx):
        loss = self._calculate_loss(batch, mode="train")
        return loss

    def validation_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="val")

    def test_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="test")

    def forward(self, X):
        return self.sequential(X)

    def _calculate_loss(self, batch, mode="train"):
        X, y = batch

        preds = self.forward(X)
        loss = F.cross_entropy(preds, y)

        # Logging to WANDB
        self.log(f"{mode}_loss", loss)
        self.log(f'{mode}_acc', metrics.accuracy(preds, y.long(), average='macro', num_classes=self.num_classes), prog_bar=True)
        self.log(f'{mode}_f1', metrics.f1_score(preds, y.long(), average='macro', num_classes=self.num_classes), prog_bar=True)
        self.log(f'{mode}_prc', metrics.precision(preds, y.long(), average='macro', num_classes=self.num_classes), prog_bar=False)
        self.log(f'{mode}_rcl', metrics.recall(preds, y.long(), average='macro', num_classes=self.num_classes), prog_bar=False)
        return loss

    def configure_optimizers(self):
        return optim.Adam(
            self.parameters(),
            lr=0.001,
            weight_decay=0.03
        )

    def train_dataloader(self):
        return data.DataLoader(
            CustomDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float()),
            batch_size=64,
            shuffle=True,
            num_workers=8
        )

    def val_dataloader(self):
        return data.DataLoader(
            CustomDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float()),
            batch_size=1,
            num_workers=8
        )

    def test_dataloader(self):
        return data.DataLoader(
            CustomDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float()),
            batch_size=1,
            num_workers=8
        )

In [ ]:
# train.py
def main(hparams):
    # wandb.finish()
    # wandb_logger = WandbLogger(project="bachelor")

    neptune_logger = NeptuneLogger(
        project="caigh/bachelor",
        api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4NjQyYTAwMy1jMmJkLTRjMzctYWQ0Zi03Y2FmYjQ1YmQ2Y2MifQ==",
        log_model_checkpoints=True,
    )

    neptune_logger.finalize('success')

    print('Loading data..')
    print(f'X shape: {X.shape[1]}')
    print(f'y shape: {y.shape[1]}')

    model = FFNetwork(
        num_features=X.shape[1],
        num_classes=y.shape[1],
        dropout=0.5
    )

    # train the model
    trainer = pl.Trainer(
        accelerator='gpu',
        strategy='dp',
        max_epochs=10,
        min_epochs=1,
        # overfit_batches=1,
        logger=neptune_logger,
        callbacks=[
            EarlyStopping(monitor="train_loss", mode="min"),
            ModelCheckpoint(dirpath='./checkpoints', filename='{epoch}-{val_loss:.2f}-{val_f1:.2f}', monitor='val_f1', save_last=True),
        ],
     )

    trainer.fit(model=model)

In [7]:
if __name__ == "__main__":
    if not in_ipython():
        root_dir = os.path.dirname(os.path.realpath(__file__))
        parser = ArgumentParser(add_help=False)
        hyperparams = parser.parse_args()

        # TRAIN
        main(hyperparams)
    else:
        main(None)

wandb: Currently logged in as: caigh. Use `wandb login --relogin` to force relogin


Global seed set to 42
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


TypeError: __init__() got an unexpected keyword argument 'checkpoint_callback'